使用pexpect模块来完成和设备的交互操作，首先来实现一个基本的登录设备的操作。

In [12]:
import pexpect
from pexpect import EOF,TIMEOUT

In [31]:
#定义ssh
def ssh_connect(username,address,port):
    ssh_command = "ssh -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -l %s %s -p %d" % (username,address,port)
    #print(ssh_command)
    return pexpect.spawn(ssh_ssh_command)


In [3]:
#定义一个设备的基本对象
class Device(object):

def __init__(self,device):
    #初始化使用一个字典，包含以下key，value
    #hostname，mgt_ip，username，password，port
    self.hostname = device.get("hostname")
    self.mgt_ip = device.get("mgt_ip")
    self.username = device.get("username")
    self.password = device.get("password")
    self.port = device.get("port",22)
    self.expect_list = []
    

IndentationError: expected an indented block (<ipython-input-3-aecdc9aa0570>, line 4)

In [6]:
def connect(self,timeout=30):
    #定义一个创建ssh的功能
    self.c = ssh_connect(self.username,self.mgt_port,self.port)
    self.c.delaybeforesend = 0.1
    return c

In [17]:
def login(self,prompt=r"[>|#|$|]\S?$"):
    #定义登录设备的基本功能
    #在登录时，遇到提示如username，或login，输入用户名
    #提示password时候，输入密码
    #登录完成后，通过>,#,$判断所在模式
    self.expect_list = []
    self.expect_list.append(r"(?i)username[:]?\s*$")
    self.expect_list.append(r"(?i)login[:]?\s*$")
    self.expect_list.append(r"(?i)password[:]?\s*$")
    self.expect_list.append(prompt)
    
    for _ in range (0,2):
        result=[]
        try:
            i = self.c.expect(self,expect_list,timeout=5)
            result.append(i)
            result.append(str(self.c.before))
            result.append(str(self.c.after))
            if i < 2:
                self.c.sendline(self.username)
            elif i == 2:
                self.c.sendline(self.password)
            elif i == 3:
                break
        except EOF:
            break
        except TIMEOUT:
            print("connect to %s timeout" %self.hostname)
            break        
    #当设备一直停在用户名或者密码的时候，那么可能用户名或密码错误
    #使用log或记录log更好，以下print输出，为了方便。
    if result[0] < 3:
        print("username or password error!")
        return result
    #大部分网络设备输出内容时，会有默认的分屏功能，类似ter len x
    #所以设置其长度为0
    self._set_terminal_length_zero()
    return result

In [18]:
#这里定义的函数，并没有完成其具体实现的代码。具体的实现，可以在子类中完成
def get_config(self):
    pass

#定义一个退出的方法。当退出时或者这个对象被删除的时候，关闭ssh进程。
def logout(self):
    if self.c:
        self.c.terminate()
def __del__(self):
    self.logout()

In [30]:
#这个子类是专门针对JUNOS的子类。
#在这个类中，需要对付类Device进行部分修改，使其符合JUNOS class JUNOS（Device）：
class JUNOS(Device):
    def __init__ (self,device):
        #先执行其父类Device的初始化方法
    super(JUNOS,self).__init__(device)
        #由于junos的设备提示符是固定的，为：“登陆的用户名@主机名”，最后使用>（操作模式）或#（配置模式）。
    self.prompt = self.username + "@" + self.hostname + "[>|#]"
    
    def login(self,prompt=""):
    #修改为了junos的提示符
        if not prompt:
            prompt = self.prompt
        return super(JUNOS,self).login(prompt)
    
    def get_config(self):
        #这个方法在父类中没有实现，在子类中，具体实现。
        #首先定义设备的提示符信息
        self.expect_list = []
        self.expect_list.append(self.prompt)
        result = []
        #通过show config命令来获取配置信息
        self.c.sendline("show config | no-more")
        try:
            i = self.c.expect(self.expect_list,timeout=5)
            if i == 0:
                result.append(0)
                result.append((self.c.before + self.c.after).decode())
            except EOF:
                pass
            except TIMEOUT:
                print("session timeout")
            return result

IndentationError: expected an indented block (<ipython-input-30-5818c71540f6>, line 6)

In [29]:
#当此代码运行时，这里就是程序的入口
if __name__ == "__main__":
    #定义测试用的设备信息
    d = {
        "hostname":"vMX-1",
        "mgt_ip":"192.168.123.33",
        "username":"admim",
        "password":"lab123",
    }
    #初始化junos这个类
    conn = JUNOS(d)
    #连接到设备
    conn.connect()
    #通过用户名，密码登录设备
    conn.login()
    #获取设备的信息
    print(conn.get_config())
    

NameError: name 'JUNOS' is not defined